# Notebook to process mass spectrometry data from the MassBank of North America (MoNA) database.

Data file is .sdf format, each chemical sample is a rdkit Chem.

In [1]:
import pathlib
import pandas as pd

from rdkit import Chem

import requests
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder